In [44]:
%matplotlib tk
import numpy as np
import pylab as pl
from sklearn.model_selection import train_test_split
from scipy.integrate import odeint

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.utils import to_categorical

/home/riccardo/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/riccardo/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/riccardo/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/riccardo/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.

In [118]:
#load the dataset 

IN_  = np.genfromtxt("Data_Lorenz/inputs")
OUT_ = np.genfromtxt("Data_Lorenz/outputs")

#normalise the data in [-1,1]

IN  = IN_/np.max(IN_)
OUT = OUT_/np.max(OUT_)

In [124]:
N = IN.shape[1]

# Build the model.
model = Sequential()
model.add(Dense(10, input_dim=N, activation='sigmoid'))
model.add(Dense(16, activation='sigmoid'))
model.add(Dense(16, activation='linear'))
model.add(Dense(N, activation='linear'))

# Compile the model.
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

In [125]:
nepoch = 10
nbatch = 1
#history = model.fit(IN_Train, OUT_Train,validation_data=(IN_Test, OUT_Test),epochs=nepoch, batch_size=nbatch) 
history = model.fit(IN, OUT,epochs=nepoch, batch_size=nbatch) 

Epoch 1/10
19990/19990 [==============================] - 23s 1ms/sample - loss: 0.0033 - acc: 0.9925
Epoch 2/10
19990/19990 [==============================] - 23s 1ms/sample - loss: 1.2735e-04 - acc: 0.9984
Epoch 3/10
19990/19990 [==============================] - 23s 1ms/sample - loss: 7.3903e-05 - acc: 0.9990
Epoch 4/10
19990/19990 [==============================] - 22s 1ms/sample - loss: 4.7402e-05 - acc: 0.9993
Epoch 5/10
19990/19990 [==============================] - 22s 1ms/sample - loss: 3.7582e-05 - acc: 0.9990
Epoch 6/10
19990/19990 [==============================] - 22s 1ms/sample - loss: 2.9618e-05 - acc: 0.9991
Epoch 7/10
19990/19990 [==============================] - 22s 1ms/sample - loss: 2.5268e-05 - acc: 0.9994
Epoch 8/10
19990/19990 [==============================] - 22s 1ms/sample - loss: 2.0751e-05 - acc: 0.9994
Epoch 9/10
19990/19990 [==============================] - 22s 1ms/sample - loss: 1.6726e-05 - acc: 0.9995
Epoch 10/10
19990/19990 [=========================

In [129]:
pl.figure(1)

pl.clf()

loss = history.history['loss']
acc  = history.history['acc']
epochs = range(1, len(loss) + 1)
pl.semilogy(epochs, loss, "r--o", lw = 1, mfc = "none" ,label='Loss')
#pl.plot(epochs, acc/np.max(acc), "b--", lw = 1, mfc = "none" ,label='Accuracy')

pl.ylabel('Training Loss')
pl.xlabel('Epochs')
pl.legend()
pl.show()

pl.savefig("Train_loss.png")

In [130]:
#compare the target with the reconstructed target

Y_NN = model.predict(IN)

pl.figure(2)

pl.subplot(3,1,1)

pl.plot(OUT[:,0],"r-",label="target")
pl.plot(Y_NN[:,0],"k--",lw=1,label="rec target")
pl.xlabel("t",fontsize=12)
pl.ylabel("X(t)",fontsize=12)
pl.legend()

pl.subplot(3,1,2)

pl.plot(OUT[:,1],"r-",label="target")
pl.plot(Y_NN[:,1],"k--",lw=1,label="rec target")
pl.xlabel("t",fontsize=12)
pl.ylabel("Y(t)",fontsize=12)

pl.subplot(3,1,3)

pl.plot(OUT[:,2],"r-",label="target")
pl.plot(Y_NN[:,2],"k--",lw=1,label="rec target")
pl.xlabel("t",fontsize=12)
pl.ylabel("Z(t)",fontsize=12)

Text(0, 0.5, 'Z(t)')

In [131]:
def lorenz(X, t, sigma, beta, rho):
    """The Lorenz equations."""
    u, v, w = X
    up = -sigma*(u - v)
    vp = rho*u - v - u*w
    wp = -beta*w + u*v
    return up, vp, wp

#choose initial condition

t = np.arange(0,100,0.01)

x0 =  IN[0,:].reshape(3,1) #5*(np.random.randn(3,1)-0.5)

u0,v0,w0 = float(x0[0]),float(x0[1]),float(x0[2])
# Lorenz paramters and initial conditions
sigma, beta, rho = 10, 2.667, 28

#integration of the Lorenz system just for reference

f = odeint(lorenz, (u0, v0, w0), t, args=(sigma, beta, rho))
x_true, y_true, z_true = f.T

#temporal integration of the neural network

ynn = np.zeros((len(t),3))
ynn[0,:] = x0.reshape(3)

for j in range(1,len(t)):
    
    y0 = model.predict(x0.T)
    ynn[j,:] = y0*np.max(OUT_) #scale factor involved in the normalisation
    x0       = y0.T

In [149]:
x_,y_,z_  = ynn[:,0],ynn[:,1],ynn[:,2]

fig = pl.figure(1)

ax = fig.gca(projection='3d')

# Make the line multi-coloured by plotting it in segments of length s which
# change in colour across the whole time series.
s = 10
n = 1000 # plot first 10 second of the simulation
#c = np.linspace(0,1,n)

ax.plot(x_[0],y_[0],z_[0],"ro")
ax.plot(x_true[0],y_true[0],z_true[0],"k-x",label="DyNN")

ax.plot(x_true, y_true, z_true, color="r",ls=":",lw=1, alpha=0.3,label="Lorenz attractor")
ax.legend()

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

for i in range(0,n-s,s):
    #ax.plot(x_true[i:i+s+1], y_true[i:i+s+1], z_true[i:i+s+1], color="k",ls="-",lw=1, alpha=0.5)
    ax.plot(x_[i:i+s+1], y_[i:i+s+1], z_[i:i+s+1], color="k",ls="-",lw=1, alpha=0.8)
    #ax.plot(Ynn[i:i+s+1,0], Ynn[i:i+s+1,1], Ynn[i:i+s+1,2], color="b",ls="-",lw=1, alpha=0.5)
    
    pl.title("t = " + str(np.round(t[i])))
    pl.pause(0.05)

    #ax.set_axis_off() 
pl.savefig("DyNN_Lorenz.png")